# rANS

对于rANS的优化主要有：  
1. interleave。
2. 在编码端使用倒数乘法代替无符号整数除法。
3. 在解码端使用alias method代替二分查找。

## 1.interleave
数据交织编码解除数据依赖。
- 效果：
  - x86  
  
|speed(MB/s)|encode|decode|
|----|----|----|
|single|1200|150|
|2way|2000|250|
|4way|2300|230|
|8way|1500|200|
|16way|1200|190|
  - arm  
  
|speed(MB/s)|encode|decode|
|----|----|----|
|single|400|58|
|2way|580|58|
|4way|730|58|
|8way|780|58|
|16way|830|55|


## 2.reciprocals
arm平台没有支持无符号整数除法的指令，需要寻找替代方法。

1. 减法  
循环减去小于当前被除数的最大除数倍数，并将商翻倍。时间复杂度为$ O(\log n)$。  
- 算法

In [3]:
def div_loop(d, r):
    '''
    Accessing the diviend and divisor by parameter D and R.
    >>> a, b = div_loop(11, 3)
    >>> a
    3
    >>> b
    2
    '''
    temp = r
    while d > (r << 1):
        r <<= 1
    q = 0
    while r >= temp: # while R can still represent one or more divisors
        q <<= 1    
        if d >= r:
            d -= r
            q += 1
        r >>= 1
    if d >= temp:
        q += 1
        d -= temp
    return q, d

In [5]:
div_loop(122, 7)

(17, 3)

2. 倒数乘法  
在 [Fabian](https://fgiesen.wordpress.com/2014/02/18/rans-with-static-probability-distributions/) 的博客中提到了将 $p$ bit的无符号整数除法转化为使用 $2p + 1$ bit 的定点乘法的倒数方法。其中提到，在除数最坏情况下（猜测是除数为或接近 $2^{32} - 1$ ）需要使用到 $2p + 1$ 个bit。  
在rANS中，我使用的是32位的被除数，如果想保证安全性与效率（因为我们希望将计算限制在64位以内），可以将被除数重新归一化到31位。但是由于我使用的除数基本不超过 $2^{7}$ ，到现在为止的运行中，没有出现错误。  
- 算法  

```c
struct SymbolStats // struct with parameters needed for reciprocal method.
{
    uint32_t freqs[N_SYM];
    uint32_t rcp_freq[N_SYM];
    uint32_t rcp_shift[N_SYM];
    
    uint32_t cum_freqs[N_SYM + 1];

    void calc_cum_freqs();
    void make_freqs();
    void make_encrcp();
};

void SymbolStats::calc_cum_freqs()
{
    // get the cumulative frequencies of symbols.
}

void SymbolStats::make_freqs()
{
    // calculate frequencies from cumulative frequencies.
}

void SymbolStats::make_encrcp()
{
    for ( int i = 0; i < N_SYM; i++) 
    {
        uint32_t freq = freqs[i];
        if (freq <= 1) 
        {
            rcp_freq[i] = ~0u;
            rcp_shift[i] = 0u;
        }
        else 
        {
            uint32_t shift = 0;
            while(freq > (1u << shift)) shift++;
            rcp_freq[i] = (uint32_t) (((1ull << (shift + 31)) + freq-1) / freq);
            rcp_shift[i] = shift - 1;
        }
    }

}

void encode(SymbolStats* syms, other parameters)
{
    // some code
    uint32_t q = ((uint32_t) (((uint64_t)x * syms->rcp_freq[symbol]) >> 32) >> syms->rcp_shift[symbol]);
    uint32_t bias = x - q * syms->freqs[symbol] + syms->cum_freqs[symbol];
    state_arr->lane[i] = (q << TOTAL_FRQ_BITS) + syms->alias_remap[bias];
}
```

In [1]:
# python version
def rcp(sigma_table):
    rshift_table = np.ones((sigma_table.shape[0], sigma_table.shape[1] - 1), dtype=np.uint32)
    rfreq_table = np.ones((sigma_table.shape[0], sigma_table.shape[1] - 1), dtype = np.uint32)
    for index, x in enumerate(sigma_table):
        rshift_arr = []
        rfreq_arr = []
        for i in range(len(x) - 1):
            freq = x[i + 1] - x[i]
            if freq <= 1:
                rfreq_arr.append((2 << 32) - 1)
                rshift_arr.append(0)
            else:
                rshift = 0
                while freq > ( 1 << rshift):
                    rshift+=1
                rfreq = (((1 << (rshift + 31)) + freq - 1) // freq) #& ((2 << 32) - 1)
                rfreq_arr.append(rfreq)
                rshift_arr.append(rshift - 1)
        print(rfreq_arr)
        rshift_table[index] = np.array(rshift_arr).astype(np.uint32)
        rfreq_table[index] = np.array(rfreq_arr).astype(np.uint32)
    return rshift_table, rfreq_table

- 效果
  - x86  
  
|speed(MB/s)|encode|decode|
|----|----|----|
|single|1200|150|
|rcp|1550|150|
|4way+rcp|2400|233|
|8way+rcp(s)|3400-3500|215|
|16way+rcp(s)|1100|192|
  - arm  
  
|speed(MB/s)|encode|decode|
|----|----|----|
|single|400|58|
|rcp|480|58|
|4way+rcp|900|59|
|8way+rcp(s)|1180|55|
|16way+rcp(s)|1050|55|


## 3.alias method

- 效果：
  - x86 
  
|speed(MB/s)|encode|decode|
|----|----|----|
|single|1200|150|
|alias+rcp|650|770|
|4way+alias(s)+rcp|300|800|
|8way+alias+rcp(s)|1300|1200|
|8way_simd|1300|2300|
|./a|750|800|
  - arm  
  
|speed(MB/s)|encode|decode|
|----|----|----|
|single|400|58|
|alias+rcp|180|458|
|4way+alias(s)|20|1050|
|4way+alias(s)+rcp|300|800|
|8way+alias+rcp(s)|340|940|
|8way+alias+rcp|310|860|